* By passenger, we mean someone in the building who is waiting to use or is currently using the elevator.
* The user of your classes should be able to customize the number of floors and passengers in the building.
* The passenger’s starting floor should be random, and they should have a random destination. (It’s up to you to determine what random should mean here.)
* Each passenger uses the elevator only once.
* Each passenger uses the elevator only once.
* (Would be nice) The program should have error checking to ensure valid user input.

In [16]:
import random
from random import randint
from copy import deepcopy

In [29]:
class Elevator(object):
    """An elevator class. 
    It has position, passengers inside and direction. 
    """

    def __init__(self, building, strategy_func, verbose, capacity = 10):
        self.current_floor = 0
        self.direction = True # True is up, False is down
        self.register_list = [] # passengers inside elevator
        self.capacity = capacity 
        self.verbose = verbose #whether we want to print or not
        self.strategy_func = strategy_func
        self.building = building # reference to building, circular reference is okay in this case 
                                 # since we don't need to worry about garbage collection when each is 
                                 # only instantiated twice
                                 # http://engineering.hearsaysocial.com/2013/06/16/circular-references-in-python/          

    def move(self):
        """Method for elevator movement.
        In every step elevator either goes a floor higher or a floor lower.
        """
        self.current_floor += 1 if self.direction else -1
    
    def enter_passengers(self):
        """Method to get in all passengers and erase them from the list
        Gets waiting passengers at current floor as long as capacity permits and they are going in elevator direction
        """
        entered = False
        for passenger in [waiter for waiter in self.building.waiting_list]: # copy so we can rm from original list
            if (passenger.start_floor == self.current_floor and len(self.register_list) < self.capacity and
                (self.current_floor == 0 or self.current_floor == self.building.num_of_floors - 1 or 
                 (self.direction and passenger.start_floor < passenger.dst_floor) or 
                 (not self.direction and passenger.start_floor > passenger.dst_floor))):
                
                entered = True
                self.register_list.append(passenger)
                self.building.waiting_list.remove(passenger)
                if self.verbose: 
                    print ("Elevator picks up passenger %d on floor %d"% (passenger.ID, passenger.start_floor))
        return entered # return value so we can track if time cost was incurred
            
    def exit_passengers(self):
        """Function to remove all passengers on their destination floor.
        It is called in run method of the building class.
        """
        exited = False
        for passenger in [reg for reg in self.register_list]:
            if passenger.dst_floor == self.current_floor:
                exited = True
                self.register_list.remove(passenger)
                passenger.end_waittime = self.building.time_step
                self.building.list_of_waittimes.append(passenger.end_waittime - passenger.start_waittime)
                if self.verbose: 
                    print("Elevator drops off passenger %d on floor %d"% (passenger.ID, passenger.dst_floor))
        return exited # return value to track if time cost was incured

In [30]:
class Passenger(object):
    """A passenger class. 
    A passenger has his or her ID number, starting floor and destination floor number
    """

    def __init__(self, ID, floorsNum):
        """Initializes values for passenger and select random floors to start
        Destination floor and starting floor differ
        """
        self.ID = ID
        self.start_floor = random.choice([0] * (floorsNum // 2) + [i for i in range(floorsNum)])
        self.dst_floor = random.choice([0] * (self.start_floor != 0) * (floorsNum // 2) + [i for i in range(floorsNum) if i != self.start_floor])
        self.start_waittime = 0
        self.end_waittime = None

In [31]:
class Building(object):
    """A building class.
    Building had number of floors, list of passengers outside the elevator and its own elevator object.
    Strategy of an elevator is set by integer, where 0 is default and 1 is another very naive strategy.
    """
    MOVE_COST_FACTOR = 2
    ENTER_COST_FACTOR = 1
    EXIT_COST_FACTOR = 1 

    def __init__(self, strategy_func, verbose = True):
        """Makes building class and addes list of passengers
        The list is sorted by starting floor value for efficacy.
        """
        self.num_of_floors = self.get_value("Please write number of floors in the building: ",\
                                   "Incorrect value. Number of floors should be integer higher than 1.", 2)

        self.passengers_num = self.get_value("Please write number of passengers: ",\
                                    "Incorrect value. Number of passengers should be non-negative integer.", 0)
        
        self.verbose = verbose
        self.elevator = Elevator(self, strategy_func, verbose)
        self.time_step = 0 
        self.list_of_waittimes = [] 
        
        self.waiting_list = []
        for i in range(self.passengers_num):
            self.waiting_list.append(Passenger(i, self.num_of_floors))
            self.waiting_list = sorted(self.waiting_list, key=lambda x: x.start_floor) 
    
    def get_value(self, message, incorret_message, minimal):
        """Method for making sure to obtain integer input
        """
        val = None
        try:
            val = int(raw_input(message))
        except ValueError:
            print incorret_message
            return self.get_value(message, incorret_message, minimal)
        if val < minimal:
            print incorret_message
            return self.get_value(message, incorret_message, minimal)
        else:
            return val

    def run(self):
        """Runs the elevator until done, in each iteration:
        1. waiting passengers enter the elevator (register_passenger)
        2. the elevator is assigned a direction value
        3. elevator moves one floor up or down
        4. passengers on destination floors leaves the elevator (cancel_passenger)
        """        
        
        while self.waiting_list or self.elevator.register_list:
            exit = self.elevator.exit_passengers()
            enter = self.elevator.enter_passengers()
            self.elevator.strategy_func(self.elevator)
            self.elevator.move()
            self.time_step += Building.MOVE_COST_FACTOR * 1  + Building.EXIT_COST_FACTOR * exit + Building.ENTER_COST_FACTOR * enter 
        
        if self.verbose:  print ("Wait time of passengers", self.list_of_waittimes)
        return self.time_step

In [36]:
# Here we define the different strategies our elevator can follow
def direction_default_strategy(self):
    """Default function of elevator work - 
    it starts with going to the roof of building, then
    comes back to the first floor.
    """
    if self.current_floor >= self.building.num_of_floors:
         self.direction = False
    elif self.current_floor <= 0:
        self.direction = True

def direction_bad_strategy(self):
    """Example function of very bad strategy. Elevator takes desired floor of the first
    passenger and elevates him to desired location, then repeats for the next passenger.
    If no passenger is in elevator it just goes to the up and down, as in default strategy.
    """
    if len(self.register_list) is 0:
        direction_default_strategy(self)
        return
    firstval = self.register_list[0].dst_floor
    if self.current_floor > firstval:
        self.direction = False
    else:
        self.direction = True

In [37]:
def main():
    """Main function"""
    building_def = Building(strategy_func = direction_default_strategy, verbose = False)
    building_bad = deepcopy(building_def)
    building_bad.elevator.strategy_func = direction_bad_strategy
    building_bad.strategy_func = direction_bad_strategy
    print ""    
    print ("Number of steps for default strategy:", building_def.run())
    print ("Number of steps for bad strategy:", building_bad.run())

if __name__ == "__main__":
    main()

Please write number of floors in the building: 100
Please write number of passengers: 100

('Number of steps for default strategy:', 1516)
('Number of steps for bad strategy:', 1516)
